In [20]:
import pandas as pd
import glob

In [8]:
# print(list(df.columns))

In [21]:
%load_ext autoreload
%autoreload 2
import glob
import numpy as np
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm, trange
from sglm.models import sglm
from sglm.features import gen_signal_df as gsd
from sglm.features import build_features as bf
from sglm.features import gen_signal_df as gsd
from sglm.features import build_features as bf

import itertools

signal_files = []
out_files = None

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:

y_col_lst_all = ['gACH', 'rDA', 'gDA',              
                 'gACH_flx_drd', 'rDA_flx_drd', 'gDA_flx_drd',
                 'gACH_flx_cht', 'rDA_flx_cht', 'gDA_flx_cht',
                 'gACH_flx_drdcht', 'rDA_flx_drdcht', 'gDA_flx_drdcht',
                 'Ch5', 'Ch6', 'GP_1', 'GP_2', 'GP_5', 'GP_6', 'SGP_1', 'SGP_2', 'SGP_5', 'SGP_6']


In [23]:

# from logging.config import _RootLoggerConfiguration

import os
from pathlib import Path

def create_folder_if_not_exists(dr):
    dr = Path(dr)
    constructed_dir = str(Path('/').resolve())
    made = False
    
    for fold in dr.parts:
        if len(fold) == 0:
            continue
        constructed_dir = str((Path(constructed_dir) / fold).resolve())
#         print('constructed_dir', constructed_dir)
        if os.path.isdir(constructed_dir):
            # print(f'Directory already exists:', constructed_dir)
            pass
        else:
            # print(f'Creating directory:', constructed_dir)
            os.mkdir(constructed_dir)
            made = True
    if made:
        print(f'Created directory:', constructed_dir)
    return

# create_folder_if_not_exists((Path.home() / 'Desktop/nada/folder2').resolve())

In [24]:
# ### Figure 1: Single side recording
figure = 'fig1'

# # Load Signal Data

# signal_files = glob.glob(f'../../data/raw/GLM_SIGNALS_WT61_*')
# signal_files += glob.glob(f'../../data/raw/GLM_SIGNALS_WT63_*')
# signal_files += glob.glob(f'../../data/raw/GLM_SIGNALS_WT64_*')

ignore_files = [
                'WT61_10152021',
                'WT61_10082021'
                ]
# for ign in ignore_files:
#     signal_files = [_ for _ in signal_files if ign not in _]

# table_files = [_.replace('GLM_SIGNALS', 'GLM_TABLE') for _ in signal_files]

# channel_definitions = {
#         ('WT61',): {'Ch1': 'gACH', 'Ch2': 'rDA'},
#         ('WT64',): {'Ch1': 'gACH', 'Ch2': 'empty'},
#         ('WT63',): {'Ch1': 'gDA', 'Ch2': 'empty'},
#     }

group_1_mice = ['WT63', 'WT64', 'WT65']
# group_1_sess = ['11082021', '11102021', '11122021', '11182021']
group_1_sess = ['11082021', '11102021', '11122021', '11162021', '11182021']
group_1_combo = ['_'.join(_) for _ in list(itertools.product(group_1_mice, group_1_sess))]

group_2_mice = ['WT66', 'WT67', 'WT68', 'WT69']
group_2_sess = ['12132021', '12152021', '12172021', '12192021']
group_2_combo = ['_'.join(_) for _ in list(itertools.product(group_2_mice, group_2_sess))]

group_3_mice = ['WT58', 'WT60', 'WT61']
group_3_sess = ['10042021', '10062021', '10082021', '10112021', '10132021', '10152021']
group_3_combo = ['_'.join(_) for _ in list(itertools.product(group_3_mice, group_3_sess))]

group_4_mice = ['WT53', 'WT54', 'WT55', 'WT56']
group_4_sess = ['09012021', '09032021', '09062021']
group_4_combo = ['_'.join(_) for _ in list(itertools.product(group_4_mice, group_4_sess))]

group_5_mice = ['WT57', 'WT59']
group_5_sess = ['10042021', '10062021', '10082021', '10112021', '10132021', '10152021']
group_5_combo = ['_'.join(_) for _ in list(itertools.product(group_5_mice, group_5_sess))]

group_6_mice = []#['WT62']
group_6_sess = []#['11082021', '11102021', '11122021', '11182021']
group_6_combo = ['_'.join(_) for _ in list(itertools.product(group_6_mice, group_6_sess))]


channel_definitions = {}
# channel_definitions = {(file_combo,): {'Ch1': 'gACH', 'Ch2': 'rDA'} for file_combo in group_1_combo}
channel_definitions.update({(file_combo,): {'Ch1': 'gDA', 'Ch5': 'gACH'} for file_combo in group_1_combo})
channel_definitions.update({(file_combo,): {'Ch1': 'gDA'} for file_combo in group_2_combo})
channel_definitions.update({(file_combo,): {'Ch1': 'gACH'} for file_combo in group_3_combo})
channel_definitions.update({(file_combo,): {'Ch1': 'gACH'} for file_combo in group_4_combo})
channel_definitions.update({(file_combo,): {'Ch5': 'gACH'} for file_combo in group_5_combo})
channel_definitions.update({(file_combo,): {'Ch1': 'gDA'} for file_combo in group_6_combo})




fig_1_signal_files_setup = group_1_combo + group_2_combo + group_3_combo + group_4_combo + group_5_combo + group_6_combo

for f in fig_1_signal_files_setup:
    glob_file = glob.glob(str(Path(f'../../data/raw/{figure}/GLM_SIGNALS_{f}*').resolve()))
    if len(glob_file) != 1:
        print('Missing file!!! ', f)
    signal_files += glob_file

Missing file!!!  WT63_11082021
Missing file!!!  WT56_09062021
Missing file!!!  WT59_10062021


In [61]:
for ign in ignore_files:
    signal_files = [_ for _ in signal_files if ign not in _]

table_files = [_.replace('GLM_SIGNALS', 'GLM_TABLE') for _ in signal_files]


channel_assignments = bf.get_rename_columns_by_file(signal_files, channel_definitions)

for file_num in trange(len(signal_files)):

    ## Load Table Data
    # signal_fn = signal_files[0]
    # table_fn = table_files[0]

    signal_path = signal_files[file_num]
    table_path = table_files[file_num]

    signal_fn = Path(signal_files[file_num]).parts[-1]
    table_fn = Path(table_files[file_num]).parts[-1]

    signal_filename_out = signal_fn.replace('GLM_SIGNALS', 'GLM_SIGNALS_INTERIM').replace('txt', 'csv')
    table_filename_out = table_fn.replace('GLM_TABLE', 'GLM_TABLE_INTERIM').replace('txt', 'csv')

    # signal_path_out = f'../../data/interim/{signal_filename_out}'
    # table_path_out = f'../../data/interim/{table_filename_out}'

    signal_path_out = signal_path.replace(r'raw', r'interim2').replace('GLM_SIGNALS', 'GLM_SIGNALS_INTERIM').replace('txt', 'csv')
    table_path_out = table_path.replace(r'raw', r'interim2').replace('GLM_TABLE', 'GLM_TABLE_INTERIM').replace('txt', 'csv')

    
    if out_files is not None and signal_path in out_files:
        signal_path_out = signal_path_out.replace(*out_files[signal_path])
        table_path_out = table_path_out.replace(*out_files[signal_path])

    create_folder_if_not_exists(str(Path('/'.join(Path(signal_path_out).parts[:-1])).resolve()))
    create_folder_if_not_exists(str(Path('/'.join(Path(table_path_out).parts[:-1])).resolve()))
    
    signal_df = pd.read_csv(signal_path)
    table_df = pd.read_csv(table_path)

    # Check for multiple copies of the same sideIn
    eq = table_df['photometrySideInIndex'].dropna()
    eq = eq[eq != 0]
    if len(eq) != eq.nunique():
        print(len(eq))
        print(eq.nunique())
        display(eq)
        print(f'Error: Multiple side ins with the same timestamp detected for {signal_path}. Continuing...')
        continue

    print(signal_path)
    
    



    signal_filename = signal_path
    table_filename = table_path
    table_index_columns = ['photometryCenterInIndex',
                            'photometryCenterOutIndex',
                            'photometrySideInIndex',
                            'photometrySideOutIndex',
                            'photometryFirstLickIndex'
                            ]
    basis_Aa_cols = ['AA', 'Aa', 'aA', 'aa', 'AB', 'Ab', 'aB', 'ab']
    trial_bounds_before_center_in = -20
    trial_bounds_after_side_out = 20


    # Load Signal Data
    signal_df = pd.read_csv(signal_filename)

    # Load Table Data
    table_df = pd.read_csv(table_filename)
    df_t2 = table_df.copy()

    # Generate Ab Labels
    df_t = gsd.generate_Ab_labels(table_df)
    assert np.all(df_t['label'].dropna() == df_t['word'].dropna())
    # print(df_t[['label', 'word']])

    # Convert Ab Labels to Indicator Variables
    ab_dummies = pd.get_dummies(df_t['label'])
    for basis_col in basis_Aa_cols:
        if basis_col not in ab_dummies.columns:
            df_t[basis_col] = 0
    df_t[ab_dummies.columns] = ab_dummies

    # Convert MATLAB 1 indexing to python 0 indexing
    df_t[table_index_columns] = gsd.matlab_indexing_to_python(df_t[table_index_columns])

    # Replace center outs that got transferred to the next timestep to be equal to center in value
    # df_t = replace_missed_center_out_indexes(df_t, verbose=1)
    df_t = df_t.copy()

    i = 0

    while True:
        
        num_inx_vals = df_t[df_t['photometryCenterOutIndex'] > 0].groupby('photometryCenterOutIndex')['hasAllPhotometryData'].count()
        # print(_, num_inx_vals.max())
        if num_inx_vals.max() == 1:
            break
        duplicated_CO_inx = df_t['photometryCenterOutIndex'] == df_t['photometryCenterOutIndex'].shift(-1)
        df_t.loc[duplicated_CO_inx, 'photometryCenterOutIndex'] = df_t.loc[duplicated_CO_inx, 'photometryCenterInIndex']

        if max_num_duplications and i > max_num_duplications:
            break
        i += 1
    

    
    # 
    for col in table_index_columns:
        df_t_tmp = df_t[gsd.get_is_relevant_trial(df_t['hasAllPhotometryData'], df_t[col])].copy()
        assert len([_ for _ in df_t_tmp.columns if col == _]) == 1, f"Error: Duplicate entries for {col}"
        assert signal_df.index.nunique() == len(signal_df.index), f"Error: Duplicate entries in signal_df.index"
        # assert signal_df.index.nunique() == len(signal_df.index), f"Error: Duplicate entries in signal_df.index"

        # print(col)
        # print('>', (df_t_tmp.set_index(col)['wasRewarded'] == df_t_tmp.set_index(col)['wasRewarded']*1).index)
        # print('> len', len(signal_df.index))
        # print('> nunique', signal_df.index.nunique())
        # print('> len b', len((df_t_tmp.set_index(col)['wasRewarded'] == df_t_tmp.set_index(col)['wasRewarded']*1).index))
        # print('> nu b', )

        # stop_flg = False

        # eq = (df_t_tmp.set_index(col)['wasRewarded'] == df_t_tmp.set_index(col)['wasRewarded']*1)
        # for e in eq.index:
        #     if len(eq.loc[[e]]) > 1:
        #         display(eq.loc[e])
        #         stop_flg = True

        # If it's not nan and it shows up in the table set it to 1 in signal df, else 0.
        signal_df[col] = (df_t_tmp.set_index(col)['wasRewarded'] == df_t_tmp.set_index(col)['wasRewarded'])*1

        # Set r to if it was rewarded.
        signal_df[f'{col}r'] = df_t_tmp.set_index(col)['wasRewarded']

        # Set nr to 1 - r
        signal_df[f'{col}nr'] = (1 - signal_df[f'{col}r'])

        # Set side in / side out word values in signal to associated values in table. Else 0 if not in table.
        if col in ['photometrySideInIndex', 'photometrySideOutIndex']: #, 'photometryCenterInIndex']:
            for basis in basis_Aa_cols:
                signal_df[col+basis] = df_t_tmp.set_index(col)[basis].fillna(0)
    
    # Trial number is the number of Center In values that have been observed shifted backwards by trial_bounds_before_center_in
    signal_df['nTrial'] = gsd.get_trial_start(signal_df['photometryCenterInIndex']).cumsum()
    # Trial end number is the number of Side Out values that have been observed shifted forwards by trial_bounds_after_side_out
    signal_df['nEndTrial'] = gsd.get_trial_end(signal_df['photometrySideOutIndex']).cumsum()
    # Difference between these two is used to indicate trial overlaps by time shifts
    signal_df['diffTrialNums'] = signal_df['nTrial'] - signal_df['nEndTrial']
    signal_df['dupe'] = False

    signal_df['wi_trial_keep'] = gsd.get_is_not_iti(signal_df)


    signal_df = signal_df[signal_df['nTrial'] > 0].fillna(0)

    # # Break down Preprocess Lynne into component parts

    # # Rename Columns
    # signal_df = bf.rename_consistent_columns(signal_df)
    
    # # print(channel_assignments.keys())
    # # print(signal_fn)
    # if signal_fn in channel_assignments:
    #     signal_df = signal_df.rename(channel_assignments[signal_fn], axis=1)

    # for y_col in y_col_lst_all:
    #     if y_col not in signal_df.columns:
    #         signal_df[y_col] = np.nan
    #         continue
    
    
    # ## Set Full Trial Reward Flags
    # signal_df['r_trial'] = (signal_df.groupby('nTrial')['photometrySideInIndexr'].transform(np.sum) > 0) * 1.0
    # signal_df['nr_trial'] = (signal_df.groupby('nTrial')['photometrySideInIndexnr'].transform(np.sum) > 0) * 1.0

    # ## Define Side Rewarded / Unrewarded Flags
    # signal_df = bf.set_port_entry_exit_rewarded_unrewarded_indicators(signal_df)

    # ## Define Side Agnostic Events
    # signal_df = bf.define_side_agnostic_events(signal_df)

    # # print('Percent of Data in ITI:', (df['nTrial'] == df['nEndTrial']).mean())

    # signal_df['spnrOff'] = ((signal_df['spnr'] == 1)&(signal_df['photometrySideInIndex'] != 1)).astype(int)
    # signal_df['spxrOff'] = ((signal_df['spxr'] == 1)&(signal_df['photometrySideOutIndex'] != 1)).astype(int)
    # spnnrOff_a = ((signal_df['spnnr'] == 1)&(signal_df['photometrySideInIndex'] != 1)).astype(int)
    # spxnrOff_a = ((signal_df['spxnr'] == 1)&(signal_df['photometrySideOutIndex'] != 1)).astype(int)

    # # If we have something listed as a rewarded "off" side entry labeled in the table as a side exit... it means it was a fast "out-in".
    # # The latter "in" should be considered an unrewarded side port "off" entry.
    # dualism_exen = ((signal_df['spnrOff'] == 1)&(signal_df['photometrySideOutIndex'] == 1)).astype(int)

    # # Unrewarded side port entries should be the combination of those simply identified by checking spnnr & the table labels +
    # # the dualism defined immediately prior. Then those dualism examples should be remoed from the "off" rewarded entries.
    # signal_df['spnnrOff'] = spnnrOff_a + dualism_exen
    # signal_df['spnrOff'] = signal_df['spnrOff'] - dualism_exen

    # signal_df['spxnrOff'] = spxnrOff_a

    

    # signal_df['slOff'] = signal_df['sl'] * signal_df['nr_trial']
    # signal_df['slOn'] = signal_df['sl'] - signal_df['slOff']


    
    # signal_df['cpnOff'] = ((signal_df['cpn'] == 1)&(signal_df['photometryCenterInIndex'] != 1)).astype(int)
    # signal_df['cpxOff'] = ((signal_df['cpx'] == 1)&(signal_df['photometryCenterOutIndex'] != 1)).astype(int)
    
    # # if (df_t['photometryCenterInIndex'] > df_t['photometryCenterOutIndex']).sum() > 0:
    # #     break

    # # import glob
    # # for csv in glob.glob('/Users/josh/Documents/Harvard/GLM/sabatinilab-glm/sglm/data/interim/fig1/GLM_SIGNALS_INTERIM_*.csv'):
    df = signal_df.copy()
    df = df[~df['dupe']]
    df['nTrial_hard'] = df['photometryCenterInIndex'].cumsum()
    df['nEndTrial_hard'] = df['photometrySideOutIndex'].cumsum().shift(1)
    df['diffTrialNums_hard'] = df['nTrial_hard'] - df['nEndTrial_hard']
    
    for col in ['photometryCenterInIndex', 'photometryCenterOutIndex',
                'photometrySideInIndex', 'photometrySideOutIndex',
                'photometrySideInIndexr', 'photometrySideOutIndexr',

                'photometrySideInIndexAA', 'photometrySideInIndexAa',
                'photometrySideInIndexaA', 'photometrySideInIndexaa',
                'photometrySideInIndexAB', 'photometrySideInIndexAb',
                'photometrySideInIndexaB', 'photometrySideInIndexab',
                ]:

        sm = df[(df['diffTrialNums_hard'] == 1)].groupby('nTrial_hard')[[col]].sum()
        sm = sm[sm[col] > 1]

        lsm = len(sm)
        if lsm > 0:
            print(csv, col, len(sm))
            break
    break

('WT63_11082021',)
('WT63_11102021',)
> GLM_SIGNALS_WT63_11102021.txt
('WT63_11122021',)
> GLM_SIGNALS_WT63_11122021.txt
('WT63_11162021',)
> GLM_SIGNALS_WT63_11162021.txt
('WT63_11182021',)
> GLM_SIGNALS_WT63_11182021.txt
('WT64_11082021',)
> GLM_SIGNALS_WT64_11082021.txt
('WT64_11102021',)
> GLM_SIGNALS_WT64_11102021.txt
('WT64_11122021',)
> GLM_SIGNALS_WT64_11122021.txt
('WT64_11162021',)
> GLM_SIGNALS_WT64_11162021.txt
('WT64_11182021',)
> GLM_SIGNALS_WT64_11182021.txt
('WT65_11082021',)
> GLM_SIGNALS_WT65_11082021.txt
('WT65_11102021',)
> GLM_SIGNALS_WT65_11102021.txt
('WT65_11122021',)
> GLM_SIGNALS_WT65_11122021.txt
('WT65_11162021',)
> GLM_SIGNALS_WT65_11162021.txt
('WT65_11182021',)
> GLM_SIGNALS_WT65_11182021.txt
('WT66_12132021',)
> GLM_SIGNALS_WT66_12132021.txt
('WT66_12152021',)
> GLM_SIGNALS_WT66_12152021.txt
('WT66_12172021',)
> GLM_SIGNALS_WT66_12172021.txt
('WT66_12192021',)
> GLM_SIGNALS_WT66_12192021.txt
('WT67_12132021',)
> GLM_SIGNALS_WT67_12132021.txt
('WT67_12152

/Users/josh/Documents/Harvard/GLM/sabatinilab-glm/sglm/data/raw/fig1/GLM_SIGNALS_WT63_11102021.txt
/Users/josh/Documents/Harvard/GLM/sabatinilab-glm/sglm/data/interim/fig1/GLM_SIGNALS_INTERIM_WT65_11102021.csv photometryCenterOutIndex 55



In [88]:
# df_t3 = df_t.copy()
# num_inx_vals = df_t3[df_t3['photometryCenterOutIndex'] > 0].groupby('photometryCenterOutIndex')['hasAllPhotometryData'].count()
# # print(_, num_inx_vals.max())
# # if num_inx_vals.max() == 1:
# #     break
# duplicated_CO_inx = (df_t3['photometryCenterOutIndex'] > df_t3['photometryCenterOutIndex'].shift(-1))&(df_t3['photometryCenterOutIndex'].shift(-1) > 0)&(df_t3['photometryCenterOutIndex'] > 0)
# df_t3.loc[duplicated_CO_inx, 'photometryCenterOutIndex'] = df_t3.loc[duplicated_CO_inx, 'photometryCenterInIndex']

In [89]:
# df_t

In [90]:
# df_t3

In [71]:
# df_t2[df_t2['photometryCenterOutIndex'].between(1240,1250)]
# df_t2[df_t2['photometryCenterOutIndex'].between(1710,1720)]
df_t2[df_t2['photometryCenterOutIndex'].between(1710,1720)]



,hasAllPhotometryData,photometryCenterInIndex,photometryCenterOutIndex,photometrySideInIndex,photometrySideOutIndex,photometryFirstLickIndex,choseLeft,choseRight,leftRewardProb,rightRewardProb,wasRewarded,word
17,1,1631,1718,1650,1692,1653,1,0,0.85,0.15,1,AA
18,1,1717,1717,1734,1775,1737,1,0,0.85,0.15,1,AA


In [68]:
# df[df['nTrial_hard']==2][[_ for _ in df.columns if 'Center' in _]]
df[df['nTrial_hard']==8][[_ for _ in df.columns if 'Center' in _]]


,photometryCenterInIndex,photometryCenterInIndexr,photometryCenterInIndexnr,photometryCenterOutIndex,photometryCenterOutIndexr,photometryCenterOutIndexnr
1716,1.0,1.0,0.0,1.0,1.0,0.0
1717,0.0,0.0,0.0,1.0,1.0,0.0
1718,0.0,0.0,0.0,0.0,0.0,0.0
1719,0.0,0.0,0.0,0.0,0.0,0.0
1720,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...
1831,0.0,0.0,0.0,0.0,0.0,0.0
1832,0.0,0.0,0.0,0.0,0.0,0.0
1833,0.0,0.0,0.0,0.0,0.0,0.0
1834,0.0,0.0,0.0,0.0,0.0,0.0


In [66]:

for col in ['photometryCenterInIndex', 'photometryCenterOutIndex',
            'photometrySideInIndex', 'photometrySideOutIndex',
            'photometrySideInIndexr', 'photometrySideOutIndexr',

            'photometrySideInIndexAA', 'photometrySideInIndexAa',
            'photometrySideInIndexaA', 'photometrySideInIndexaa',
            'photometrySideInIndexAB', 'photometrySideInIndexAb',
            'photometrySideInIndexaB', 'photometrySideInIndexab',]:

    sm = df[(df['diffTrialNums_hard'] == 1)].groupby('nTrial_hard')[[col]].sum()
    sm = sm[sm[col] > 1]

    lsm = len(sm)
    if lsm > 0:
        print(csv, col, len(sm))
        print(sm)

/Users/josh/Documents/Harvard/GLM/sabatinilab-glm/sglm/data/interim/fig1/GLM_SIGNALS_INTERIM_WT65_11102021.csv photometryCenterOutIndex 55
             photometryCenterOutIndex
nTrial_hard                          
2.0                               2.0
8.0                               2.0
11.0                              2.0
17.0                              2.0
20.0                              2.0
25.0                              2.0
28.0                              2.0
32.0                              2.0
34.0                              2.0
41.0                              2.0
45.0                              2.0
55.0                              2.0
64.0                              2.0
78.0                              2.0
83.0                              2.0
89.0                              2.0
100.0                             2.0
111.0                             2.0
123.0                             2.0
126.0                             2.0
136.0                    

In [ ]:
# # import glob
#     # for csv in glob.glob('/Users/josh/Documents/Harvard/GLM/sabatinilab-glm/sglm/data/interim/fig1/GLM_SIGNALS_INTERIM_*.csv'):
#     df = pd.read_csv(csv)
#     df = df[~df['dupe']]
#     df['nTrial_hard'] = df['photometryCenterInIndex'].cumsum()
#     df['nEndTrial_hard'] = df['photometrySideOutIndex'].cumsum().shift(1)
#     df['diffTrialNums_hard'] = df['nTrial_hard'] - df['nEndTrial_hard']
    
#     for col in ['photometryCenterInIndex', 'photometryCenterOutIndex',
#                 'photometrySideInIndex', 'photometrySideOutIndex',
#                 'photometrySideInIndexr', 'photometrySideOutIndexr',

#                 'photometrySideInIndexAA', 'photometrySideInIndexAa',
#                 'photometrySideInIndexaA', 'photometrySideInIndexaa',
#                 'photometrySideInIndexAB', 'photometrySideInIndexAb',
#                 'photometrySideInIndexaB', 'photometrySideInIndexab',
#                 ]:

#         sm = df[(df['diffTrialNums_hard'] == 1)].groupby('nTrial_hard')[[col]].sum()
#         sm = sm[sm[col] > 1]

#         lsm = len(sm)
#         if lsm > 0:
#             print(csv, col, len(sm))


In [36]:
# import pandas as pd
# df = pd.read_csv('/Users/josh/Documents/Harvard/GLM/sabatinilab-glm/sglm/data/raw/fig1/GLM_TABLE_WT63_11102021.txt')
# tmp = (df['photometryCenterOutIndex'] > 0)&((df['photometryCenterOutIndex'] >= df['photometryCenterOutIndex'].shift(-1))|(df['photometryCenterOutIndex'].shift(1) >= df['photometryCenterOutIndex']))
# inx = tmp[tmp].index

In [37]:
# print(len(list(inx)))

In [38]:
# with pd.option_context('display.max_rows', 1000):
#     df.loc[inx]

In [37]:
# df_t

In [4]:
# df.groupby('nEndTrial_hard')['photometryCenterInIndex'].sum()

In [ ]:
a

In [5]:
# df['diffTrialNums'].value_counts()

1.0    16125
0.0     6145
2.0     4083
Name: diffTrialNums, dtype: int64